In [0]:
from azure.storage.blob import BlobServiceClient
from datetime import datetime
from io import StringIO
import pandas as pd
from pandas import DataFrame
import ast

In [0]:
def criar_dataFrame(dados: DataFrame) -> DataFrame:
    """Criando DataFrame apartir dos dados"""
    df = pd.read_csv(StringIO(dados))
    df['dados'] = df['dados'].apply(ast.literal_eval)
    return pd.json_normalize(df['dados'])

In [0]:
def converter_para_minutos(duracao: str) -> float:
    """Converte a duração do vídeo para minutos"""
    duracao = duracao.split('H')
    horas = int(duracao[0]) if len(duracao) > 1 else 0 
    duracao = duracao[1] if len(duracao) > 1 else duracao[0]

    duracao = duracao.split('M')
    minutos = int(duracao[0]) if len(duracao) > 1 else 0 
    duracao = duracao[1] if len(duracao) > 1 else duracao[0]

    duracao = duracao.split('S')
    segundos = int(duracao[0]) if len(duracao) > 1 else 0

    return round(horas * 60 + minutos + segundos / 60, 2)

In [0]:
def tratamento_dados(df: DataFrame) -> DataFrame:
    """Tratando Dados"""
    #Excluindo Colunas 
    colunas_para_excluir = ['etag', 'snippet.channelId', 'snippet.thumbnails.default.url','snippet.thumbnails.default.width',	'snippet.thumbnails.default.height','snippet.thumbnails.medium.url','snippet.thumbnails.medium.width',	'snippet.thumbnails.medium.height',	'snippet.thumbnails.high.url', 'snippet.thumbnails.high.width', 'snippet.thumbnails.high.height', 'snippet.thumbnails.standard.url', 'snippet.thumbnails.standard.width','snippet.thumbnails.standard.height', 'snippet.thumbnails.maxres.width', 'snippet.thumbnails.maxres.height', 'snippet.categoryId',	'snippet.liveBroadcastContent', 'contentDetails.caption','contentDetails.licensedContent','contentDetails.projection', 'statistics.favoriteCount',  'snippet.defaultLanguage',	'contentDetails.regionRestriction.allowed', 'snippet.localized.title', 'snippet.localized.description', 'contentDetails.dimension']

    df = df.drop(colunas_para_excluir, axis=1)

    #Tratando a coluna 'kind'
    df['kind'] = df['kind'].apply(lambda x: x.replace('#', ' '))

    #Criando coluna dia e Horario da publicação
    df[['dia', 'horario']] = df['snippet.publishedAt'].str.split('T', n = 1, expand = True)
    df = df.drop('snippet.publishedAt', axis = 1)

    #Criando uma coluna chamada link com o link do video
    df['link'] = df['id'].apply(lambda x: 'www.youtube.com/watch?v=' + x)

    #Tratando a duração do video, padronizando todos em minutos
    df['contentDetails.duration'] = df['contentDetails.duration'].apply(lambda x: x.replace('PT', ''))
    df['contentDetails.duration'] = df['contentDetails.duration'].apply(converter_para_minutos)

    #Tratando a coluna horario
    df['horario'] = df['horario'].apply(lambda x: x.replace('Z', ''))

    #Renomeando as colunas 
    mapeamento_colunas = {
    'kind': 'tipo',
    'snippet.title': 'titulo',
    'snippet.description': 'descricao', 
    'snippet.thumbnails.maxres.url':'thumbnails_png', 
    'snippet.channelTitle': 'canal', 
    'snippet.tags': 'tags', 
    'snippet.defaultAudioLanguage':'idioma', 
    'contentDetails.duration':'duracao', 
    'contentDetails.definition': 'definicao', 
    'statistics.viewCount': 'visualizacoes', 
    'statistics.likeCount': 'likes',
    'statistics.commentCount': 'comentarios', 
    'contentDetails.regionRestriction.blocked': 'Regioes_bloqueadas'
    }

    df.rename(columns=mapeamento_colunas, inplace=True)

    return df 


In [0]:
if __name__ == '__main__':

    # Configurações
    nome_conta = "etldatarafael"
    key_conta = "QZA1D2QY30SNBOlXTLfG8RY8IeSjfx6T4Jr2NHTJIAo16WmImq0nLQswh0zx2iTrBY4diUArdAXO+ASt4NM94A=="
    nome_container = "bronze"
    nome_arquivo = 'data_' + datetime.now().strftime("%Y%m%d") + '.csv'

    # Conectar ao Blob Storage Bronze
    blob_service_client = BlobServiceClient(account_url=f"https://{nome_conta}.blob.core.windows.net", credential=key_conta)
    container_client = blob_service_client.get_container_client(nome_container)
    cliente_blob = container_client.get_blob_client(nome_arquivo)
    
    #Baixando dados
    dados = cliente_blob.download_blob().readall().decode('utf-8')

    #Criando DataFrame
    df = criar_dataFrame(dados)

    #Tratando DataFrame
    df = tratamento_dados(df)


    nome_container = "silver"
    container_client = blob_service_client.get_container_client(nome_container)

    # Verifica se o arquivo já existe no Blob Storage
    blob_client = container_client.get_blob_client(nome_arquivo)
    if blob_client.exists():
        # Excluindo se o arquivo ja existir 
        blob_client.delete_blob()

    new_content = df.to_csv(index=False)

    # Converter o conteúdo para bytes e fazer o upload para o Blob Storage
    blob_client.upload_blob(StringIO(new_content).read().encode('utf-8'))


In [0]:
df.head()

,tipo,id,titulo,descricao,thumbnails_png,canal,tags,idioma,duracao,definicao,visualizacoes,likes,comentarios,Regioes_bloqueadas,dia,horario,link
0,youtube video,k_5khB5C28E,11º TÍTULO DO TIMÃO! | CORINTHIANS 1 X 0 CRUZE...,Assista a 37 jogos AO VIVO E EXCLUSIVOS da Cop...,https://i.ytimg.com/vi/k_5khB5C28E/maxresdefau...,CazéTV,"[CazéTV, Cazé TV, Casimiro ao vivo, Casimiro, ...",pt-BR,7.42,hd,293822,11994,1497,NaN,2024-01-25,20:37:32,www.youtube.com/watch?v=k_5khB5C28E
1,youtube video,1MpUldfp43Q,PALMEIRAS 3 X 2 INTER DE LIMEIRA | MELHORES MO...,Confira os melhores momentos da vitória de 3 a...,https://i.ytimg.com/vi/1MpUldfp43Q/maxresdefau...,Paulistão,"[Futebol Paulista, Melhores Momentos Paulistão...",pt,6.02,hd,237524,5726,213,[US],2024-01-25,03:13:51,www.youtube.com/watch?v=1MpUldfp43Q
2,youtube video,dPXgN_9l0aM,BOAVISTA X BOTAFOGO | CAMPEONATO CARIOCA | AO ...,#boavista #botafogo #campeonatocarioca \n\nJ...,https://i.ytimg.com/vi/dPXgN_9l0aM/maxresdefau...,Canal GOAT,"[campeonato carioca, campeonato estadual, cari...",pt,161.58,hd,903558,35921,30,NaN,2024-01-25,02:47:19,www.youtube.com/watch?v=dPXgN_9l0aM
3,youtube video,5f4zOWDj1Mw,Construi uma BASE INTEIRA só pro meu PET em 10...,😍😍 Compre agora mesmo o seu RedMagic e jogue c...,https://i.ytimg.com/vi/5f4zOWDj1Mw/maxresdefau...,Fixz,"[fixz, craft fixz, creative squad 2, creative ...",pt,29.75,hd,124874,6388,186,NaN,2024-01-25,14:00:04,www.youtube.com/watch?v=5f4zOWDj1Mw
4,youtube video,4wulgy2d_YA,BARCELONA 2 X 1 VITÓRIA | PARTIDA COMPLETA | #...,Barcelona e Vitória se enfrentam no #BaianãoNa...,https://i.ytimg.com/vi/4wulgy2d_YA/maxresdefau...,TVE Bahia,"[tve, tve bahia, tve da bahia, tv e, tve agora...",pt,148.92,hd,436095,13648,35,NaN,2024-01-25,02:52:52,www.youtube.com/watch?v=4wulgy2d_YA
